In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

# Pair Programming Random Forest

## Ana C y Ana G

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

En pairprogramming anteriores ajustastéis vuestro datos a una regresión lineal y a un Decision Tree. El objetivo de hoy es:
-  Ajustar el modelo a un Random Forest
-  Extraer las métricas
-  Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)

In [2]:
df = pd.read_csv("../data/08-primas_categorizadas_normalizadas.csv", index_col = 0)
df.head(2)

,age,sex,bmi,children,smoker,region,charges_Sklearn,fumadores,mujeres,hombres,northeast,northwest,southeast,southwest
0,-1.387916,female,-0.382685,-0.885991,yes,southwest,0.836552,1,1,0,0.0,0.0,0.0,1.0
1,-1.459488,male,0.601634,-0.069805,no,southeast,0.032037,0,0,1,0.0,0.0,1.0,0.0


In [3]:
df.shape[0]*0.2

212.8

In [4]:
df2 = pd.read_csv("../data/09-primas-altas.csv", index_col = 0)
df2.head(2)

,age,sex,bmi,children,smoker,region,charges,fumadores,sex_female,sex_male
3,33,male,22.705,0,no,northwest,21984.47061,0,0,1
9,60,female,25.840,0,no,northwest,28923.13692,0,1,0


In [5]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X = df.select_dtypes(np.number).drop("charges_Sklearn", axis = 1)
y = df["charges_Sklearn"]

In [6]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# recordamos que lo primero que hacíamos era crear un diccionario con los hiperparámetros que queríamos probar

param = {"max_depth": [2,4,6], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3,4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [80, 90, 100],
        "min_samples_leaf": [10,50,100]} 

In [8]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [9]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [80, 90, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [10]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=6, max_features=4, min_samples_leaf=10,
                      min_samples_split=80)

In [11]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [12]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [13]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results_bajas = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
dt_results_bajas

,MAE,MSE,RMSE,R2,set,modelo
0,0.085049,0.020473,0.143083,0.677782,test,Random Forest
1,0.074928,0.015209,0.123325,0.755108,train,Random Forest


In [16]:
# igual que en la clase de decision tree, juntemos estos resultados con los obtenidos en los modelos que hemos hecho hasta ahora
# lo primero que tenemos que hacer es cargar el csv con todos los resultados que guardamos en el jupyter anterior

df_LR_DT_results_bajas = pd.read_csv("../data/11-df_decision_results_bajas.csv", index_col = 0)
df_LR_DT_results_bajas




,MAE,MSE,RMSE,R2,set,modelo
0,0.073359,0.033802,0.183854,0.467991,test,Decission Tree I
1,0.000780,0.000259,0.016081,0.995836,train,Decission Tree I
0,0.096724,0.021730,0.147412,0.657989,test,Decision tree II
1,0.093899,0.018988,0.137796,0.694264,train,Decision tree II


In [17]:
df_bajas=pd.read_csv("../data/10-resultados_metricas.csv",index_col=0)

In [18]:
# concatenamos el csv anterior con los resultados obtenidos en este modelo

df_todos_resultados_bajas = pd.concat([dt_results_bajas, df_LR_DT_results_bajas, df_bajas], axis = 0)
df_todos_resultados_bajas

,MAE,MSE,RMSE,R2,set,modelo
0,0.085049,0.020473,0.143083,0.677782,test,Random Forest
1,0.074928,0.015209,0.123325,0.755108,train,Random Forest
0,0.073359,0.033802,0.183854,0.467991,test,Decission Tree I
1,0.000780,0.000259,0.016081,0.995836,train,Decission Tree I
0,0.096724,0.021730,0.147412,0.657989,test,Decision tree II
1,0.093899,0.018988,0.137796,0.694264,train,Decision tree II
0,0.061607,0.016929,0.130112,0.733553,test,Linear Regresion
1,0.056192,0.013757,0.117289,0.778495,train,LinearRegression


## A la vista de los resultados, nos quedamos con el modelo de regresion lineal, que tiene el mayoer valor predictivo y sin overfitting.

---

## Primas altas

In [19]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X2 = df2.select_dtypes(np.number).drop("charges", axis = 1)
y2 = df2["charges"]

In [20]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.2, random_state = 42)

In [21]:
# recordamos que lo primero que hacíamos era crear un diccionario con los hiperparámetros que queríamos probar

param2 = {"max_depth": [2,4, 6], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3,4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [22]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [23]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf2.fit(x_train2, y_train2)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [24]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestRegressor(max_depth=6, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [25]:
y_pred_test_rf2= bosque2.predict(x_test2)
y_pred_train_rf2= bosque2.predict(x_train2)

In [26]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results_altas = metricas(y_test2, y_train2, y_pred_test_rf2, y_pred_train_rf2, "Random Forest")
dt_results_altas

,MAE,MSE,RMSE,R2,set,modelo
0,3034.577473,2.215834e+07,4707.265070,0.785418,test,Random Forest
1,2386.383625,1.289841e+07,3591.435509,0.855423,train,Random Forest


In [27]:
# igual que en la clase de decision tree, juntemos estos resultados con los obtenidos en los modelos que hemos hecho hasta ahora
# lo primero que tenemos que hacer es cargar el csv con todos los resultados que guardamos en el jupyter anterior

df_LR_DT_results_altas = pd.read_csv("../data/11-df_decision_results_altas.csv", index_col = 0)
df_LR_DT_results_altas




,MAE,MSE,RMSE,R2,set,modelo
0,2953.702660,3.247707e+07,5698.865839,0.685491,test,Decission Tree I
1,0.000000,0.000000e+00,0.000000,1.000000,train,Decission Tree I
0,3869.992281,3.003876e+07,5480.762546,0.709104,test,Decision tree II
1,3306.045243,2.004349e+07,4476.996074,0.775334,train,Decision tree II


In [28]:
df_altas=pd.read_csv("../data/10-resultados_metricas_altas.csv", index_col=0)

In [29]:
# concatenamos el csv anterior con los resultados obtenidos en este modelo

df_todos_resultados_altas = pd.concat([dt_results_altas, df_LR_DT_results_altas,df_altas], axis = 0)
df_todos_resultados_altas

,MAE,MSE,RMSE,R2,set,modelo
0,3034.577473,2.215834e+07,4707.265070,0.785418,test,Random Forest
1,2386.383625,1.289841e+07,3591.435509,0.855423,train,Random Forest
0,2953.702660,3.247707e+07,5698.865839,0.685491,test,Decission Tree I
1,0.000000,0.000000e+00,0.000000,1.000000,train,Decission Tree I
0,3869.992281,3.003876e+07,5480.762546,0.709104,test,Decision tree II
1,3306.045243,2.004349e+07,4476.996074,0.775334,train,Decision tree II
0,6753.719242,8.026782e+07,8959.230964,0.222684,test,Linear Regresion
1,6612.818781,6.994785e+07,8363.483322,0.215961,train,LinearRegression


## En este caso, vemos una mejoria notable en el modelo, tanto a nivel predictivo como de ajuste, sin duda, nos quedamos con el modelo obtenido por Random Forest